In [1]:
# Without autoencoders chech anomalies
import h2o
#import thread
import pandas as pd
import numpy as np
#from dataprocessor import Filter
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
#from parser import DataFrameParser
#from presenting import Chart
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [2]:
# Initialize h2o server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.10" 2021-01-19 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.10+8-LTS-162); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.10+8-LTS-162, mixed mode)
  Starting server from /Users/edison/opt/anaconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/18/s0tmgl2s2sz19j6tcs0sqrcw0000gn/T/tmp0fwkrzpg
  JVM stdout: /var/folders/18/s0tmgl2s2sz19j6tcs0sqrcw0000gn/T/tmp0fwkrzpg/h2o_edison_started_from_python.out
  JVM stderr: /var/folders/18/s0tmgl2s2sz19j6tcs0sqrcw0000gn/T/tmp0fwkrzpg/h2o_edison_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Singapore
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_edison_3o9k3w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
# Data
df_train = pd.read_csv("CMaps/train_FD001.txt", sep=" ", header=None)
df_test = pd.read_csv("CMaps/test_FD001.txt", sep=" ", header=None)
df_rul = pd.read_csv("CMaps/RUL_FD001.txt", sep=" ", header=None)

In [4]:
print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")
print(f"RUL: {df_rul.shape}")

Train: (20631, 28)
Test: (13096, 28)
RUL: (100, 2)


In [5]:
columns = ['Engine_ID', 'Cycle',
          'ALT', 'Mach', 'TRA',
          'T2', 'T24', 'T30', 'T50',
           'P2', 'P15', 'P30',
           'Nf', 'Nc', 'epr', 'Ps30', 'phi',
           'NRf', 'NRc', 'BPR', 'farB',
           'htBleed', 'Nf_dmd', 'PCNfR_dmd',
           'W31', 'W32', "SD_22", "SD_23"
          ]

In [6]:
df_train.columns = columns
df_test.columns = columns

In [7]:
# Count number of unique engines
print(f"df_train unique engines: {df_train['Engine_ID'].nunique()}")
print(f"df_test unique engines: {df_test['Engine_ID'].nunique()}")

df_train unique engines: 100
df_test unique engines: 100


In [8]:
print(f"df_rul engines count: {len(df_rul)}")

df_rul engines count: 100


In [9]:
#function for preparing training data and forming a RUL column with information about the remaining
# before breaking cycles
def prepare_train_data(data, factor = 0):
    df = data.copy()
    fd_RUL = df.groupby('Engine_ID')['Cycle'].max().reset_index()
    fd_RUL = pd.DataFrame(fd_RUL)
    fd_RUL.columns = ['Engine_ID','max']
    df = df.merge(fd_RUL, on=['Engine_ID'], how='left')
    df['RUL'] = df['max'] - df['Cycle']
    df.drop(columns=['max'],inplace = True)
    
    return df[df['Cycle'] > factor]

In [10]:
df_train = prepare_train_data(df_train, factor = 0)

In [11]:
# Add RUL to df_test

df_test = df_test.groupby('Engine_ID').last().reset_index()
print(len(df_test))

# Add max RUL to df_test
# Make list of df_rul
max_rul = df_rul[0].tolist() #len = 100

# Create actual RUL column: MAX_RUL - Cycle
df_test["RUL"] = max_rul

100


In [12]:
display(df_train)

,Engine_ID,Cycle,ALT,Mach,TRA,T2,T24,T30,T50,P2,...,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32,SD_22,SD_23,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,8.4956,0.03,397,2388,100.0,38.49,22.9735,NaN,NaN,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,8.5139,0.03,395,2388,100.0,38.30,23.1594,NaN,NaN,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,8.5646,0.03,398,2388,100.0,38.44,22.9333,NaN,NaN,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,8.5389,0.03,395,2388,100.0,38.29,23.0640,NaN,NaN,1


In [13]:
all_columns = list(df_train.columns)
removing_columns = ['Engine_ID', 'Cycle', 'RUL', 'ALT', 'Mach', 'TRA']
selected_columns = [x for x in all_columns if x not in removing_columns]

In [14]:
def filterData(panda_frame, columns, removal_method, threshold):
    # Anomaly index container
    rm_index = []

    # Select anomaly removal type
    if removal_method == "iqr":
        for column in columns:
            series = panda_frame[column]
            anomaly = iqr(series, threshold)
            rm_index.extend(anomaly)
    elif removal_method == "threesigma":
        for column in columns:
            series = panda_frame[column]
            anomaly = iqr(series, threshold)
            rm_index.extend(anomaly)

    # Sort indexes
    rm_index.sort()
    anomaly_series = list(set(rm_index))

    # Remove anomalies
    p_filtered = panda_frame.drop(panda_frame.index[anomaly_series])
    return p_filtered


def iqr(series, threshold=3):
    '''
    Identify anomalies according to Inner-Quartile Range
    IQR Rule
    ----------------
    Q25 = 25 th percentile
    Q75 = 75 th percentile
    IQR = Q75 - Q25 Inner quartile range
    if abs(x-Q75) > 1.5 * IQR : A mild outlier
    if abs(x-Q75) > 3.0 * IQR : An extreme outlier
    :param series: input data array
    :param threshold: 1.5 mild, 3 extreme
    :return: Index array of where anomalies are
    '''

    series = np.array(list(series))
    q25 = np.percentile(series, 25)
    q75 = np.percentile(series, 75)
    iqr = q75 - q25

    anomaly_indexes = []
    for i in range(series.size):
        if (series[i] - q75) > threshold * iqr:
            anomaly_indexes.append(i)

    return anomaly_indexes

In [15]:
filtered = filterData(panda_frame=df_train, columns=selected_columns, removal_method="iqr", threshold=3)

In [16]:
filtered.shape

(19812, 29)

In [17]:
hTrain = h2o.H2OFrame(filtered)
hTrain.set_names(list(df_train.columns))

Parse progress: |█████████████████████████████████████████████████████████| 100%


Engine_ID,Cycle,ALT,Mach,TRA,T2,T24,T30,T50,P2,P15,P30,Nf,Nc,epr,Ps30,phi,NRf,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32,SD_22,SD_23,RUL
1,1,-0.0007,-0.0004,100,518.67,641.82,1589.7,1400.6,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.419,nan,nan,191
1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100,39,23.4236,nan,nan,190
1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.2,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,nan,nan,189
1,4,0.0007,0,100,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,nan,nan,188
1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,21.61,554,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.8,8.4294,0.03,393,2388,100,38.9,23.4044,nan,nan,187
1,6,-0.0043,-0.0001,100,518.67,642.1,1584.47,1398.37,14.62,21.61,554.67,2388.02,9049.68,1.3,47.16,521.68,2388.03,8132.85,8.4108,0.03,391,2388,100,38.98,23.3669,nan,nan,186
1,7,0.001,0.0001,100,518.67,642.48,1592.32,1397.77,14.62,21.61,554.34,2388.02,9059.13,1.3,47.36,522.32,2388.03,8132.32,8.3974,0.03,392,2388,100,39.1,23.3774,nan,nan,185
1,8,-0.0034,0.0003,100,518.67,642.56,1582.96,1400.97,14.62,21.61,553.85,2388,9040.8,1.3,47.24,522.47,2388.03,8131.07,8.4076,0.03,391,2388,100,38.97,23.3106,nan,nan,184
1,9,0.0008,0.0001,100,518.67,642.12,1590.98,1394.8,14.62,21.61,553.69,2388.05,9046.46,1.3,47.29,521.79,2388.05,8125.69,8.3728,0.03,392,2388,100,39.05,23.4066,nan,nan,183
1,10,-0.0033,0.0001,100,518.67,641.71,1591.24,1400.46,14.62,21.61,553.59,2388.05,9051.7,1.3,47.03,521.79,2388.06,8129.38,8.4286,0.03,393,2388,100,38.95,23.4694,nan,nan,182


In [18]:
#hValidate = h2o.H2OFrame(pValidate)
#hValidate.set_names(list(pValidate.columns))

In [19]:
hTest = h2o.H2OFrame(df_test)
hTest.set_names(list(df_test.columns))

Parse progress: |█████████████████████████████████████████████████████████| 100%


Engine_ID,Cycle,ALT,Mach,TRA,T2,T24,T30,T50,P2,P15,P30,Nf,Nc,epr,Ps30,phi,NRf,NRc,BPR,farB,htBleed,Nf_dmd,PCNfR_dmd,W31,W32,SD_22,SD_23,RUL
1,31,-0.0006,0.0004,100,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.4,1.3,47.23,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100,38.81,23.3552,nan,nan,112
2,49,0.0018,-0.0001,100,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.1,9044.77,1.3,47.67,521.74,2388.09,8126.9,8.4505,0.03,391,2388,100,38.81,23.2618,nan,nan,98
3,126,-0.0016,0.0004,100,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,47.88,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100,38.93,23.274,nan,nan,69
4,106,0.0012,0.0004,100,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.3,1.3,47.65,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100,38.58,23.2581,nan,nan,82
5,98,-0.0013,-0.0004,100,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.1,9053.99,1.3,47.46,521,2388.15,8125.74,8.4362,0.03,394,2388,100,38.75,23.4117,nan,nan,91
6,105,0.0076,-0.0003,100,518.67,643.05,1586.94,1404.49,14.62,21.61,553.73,2388.14,9055.83,1.3,47.51,521.56,2388.15,8139.02,8.4452,0.03,393,2388,100,38.91,23.3269,nan,nan,93
7,160,0.0016,-0.0001,100,518.67,642.1,1589.59,1413.57,14.62,21.61,553.42,2388.04,9070.99,1.3,47.32,522.32,2388.07,8149.19,8.4028,0.03,393,2388,100,38.91,23.2763,nan,nan,91
8,166,0.0016,-0.0005,100,518.67,642.59,1591.91,1413.89,14.62,21.61,553.51,2388.14,9049.96,1.3,47.59,520.53,2388.12,8127.84,8.4744,0.03,393,2388,100,38.73,23.2465,nan,nan,95
9,55,-0.0003,0.0004,100,518.67,642.27,1593.61,1410.27,14.62,21.61,552.58,2388.13,9057.57,1.3,47.58,521.65,2388.16,8134.76,8.4293,0.03,392,2388,100,38.78,23.3515,nan,nan,111
10,192,-0.0018,0.0004,100,518.67,643,1589.5,1398.99,14.62,21.61,552.88,2388.11,9060.88,1.3,47.59,521.82,2388.08,8141.91,8.4194,0.03,393,2388,100,38.79,23.2956,nan,nan,96


In [20]:
training_columns = list(df_train.columns)
training_columns.remove('Engine_ID')
training_columns.remove('Cycle')
training_columns.remove('RUL')

bad_cols = ['Nf_dmd', 'TRA', 'P2', 'farB', 'PCNfR_dmd', 'epr', 'T2', 'SD_23', 'SD_22']
for i in bad_cols:
    training_columns.remove(i)

response_column = 'RUL'

In [21]:
model = H2ODeepLearningEstimator(epochs=100, loss='Automatic', activation='RectifierWithDropout', distribution='gamma', hidden=[200])
model.train(x=training_columns, y=response_column, training_frame=hTrain)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [22]:
print(model.model_performance(test_data=hTest))


ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 753.7054136363492
RMSE: 27.45369581015185
MAE: 20.09211917829768
RMSLE: 0.3604358188384623
Mean Residual Deviance: 10.248546485447076



In [23]:
predict = model.predict(test_data=hTest)
predict = h2o.as_list(predict, use_pandas=True)
predict = predict["predict"].tolist()

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [24]:
y_pred = np.array(predict)
y_test = np.array(df_test["RUL"].tolist())

In [25]:
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)**0.5
r2 = r2_score(y_test, y_pred)
print('Using Autoencoder, MAE is %0.2f' %  mae)  
print('Using Autoencoder, RMSE is %0.2f' %  rmse)   
print('Using Autoencoder, R-Squared is %0.2f' %  r2)  

Using Autoencoder, MAE is 20.09
Using Autoencoder, RMSE is 27.45
Using Autoencoder, R-Squared is 0.56
